In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
import numpy as np
import pandas as pd
import json
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder

In [9]:
tf.__version__

'2.18.0'

In [10]:
news_path = "/content/drive/MyDrive/Colab Notebooks/NLP/2/08-Word Cluste/news.xlsx"
train_df = pd.read_excel(news_path)
train_df.head()

,Date,Title,Summary,Body,Tag,cat
0,تاریخ انتشار:\n۲۹ اسفند ۱۳۹۷ - ۲۰:۴۷,قرار گرفتن فوتبال ساحلی ایران در سید دوم جام ج...,در رده بندی جدید کنفدراسیون فوتبال آسیا، تیم م...,به گزارش\nخبرنگار فوتبال و فوتسال\n\nگروه ورزش...,فدراسیون فوتبال\n،\nاخبار ورزشی,sports
1,تاریخ انتشار:\n۲۹ اسفند ۱۳۹۷ - ۲۰:۳۷,تبریک اینستاگرامی کی روش به مناسبت عید نوروز,سرمربی سابق تیم ملی فوتبال کشورمان با انتشار پ...,به گزارش\nخبرنگار فوتبال و فوتسال\n\nگروه ورزش...,تیم ملی فوتبال\n،\nکارلوس کی روش\n،\nاخبار ورزشی,sports
2,تاریخ انتشار:\n۰۱ فروردین ۱۳۹۸ - ۱۱:۰۵,تیم ملی فوتبال امید ایران - ترکمنستان؛ آغاز ما...,تیم‌های ملی فوتبال امید ایران و ترکمنستان، فرد...,به گزارش\nخبرنگار فوتبال و فوتسال\n\nگروه ورزش...,اخبار ورزشی\n،\nفدراسیون فوتبال,sports
3,تاریخ انتشار:\n۲۹ اسفند ۱۳۹۷ - ۲۱:۰۵,اولین تمرین پرسپولیس در سال ۹۸,اولین تمرین در سال جدید ۳ فروردین در ورزشگاه ش...,به گزارش\nگروه ورزش باشگاه خبرنگاران جوان\n، ب...,باشگاه پرسپولیس\n،\nاخبار ورزشی,sports
4,تاریخ انتشار:\n۲۹ اسفند ۱۳۹۷ - ۱۸:۲۶,اسلامیان: قصدمان پرداخت نکردن پول کی روش نیست/...,عضو هیئت رئیسه فدراسیون فوتبال گفت: پول کی روش...,به گزارش\nخبرنگار فوتبال و فوتسال\n\nگروه ورزش...,تیم ملی فوتبال ایران\n،\nاخبار ورزشی\n،\nکارلو...,sports


In [11]:
train_df = train_df[train_df.Body.duplicated()==False]

In [12]:
train_df.shape

(5438, 6)

In [13]:
train_df.cat.value_counts()

,count
cat,
political,3000
comercial,1496
sports,942


In [14]:
sw_path = "/content/drive/MyDrive/Colab Notebooks/NLP/2/08-Word Cluste/stopwords.txt"
stopwords = []
file = open(sw_path, encoding='utf-8').read()
[stopwords.append(x) for x in file.split()]
stopwords = set(stopwords)

In [15]:
def remove_general_stopwords(text):
    filtered_tokens = [token for token in text.split() if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [16]:
num_classes = len(train_df.cat.unique())
num_classes

3

In [17]:
Y = train_df['cat']
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y

array([2, 2, 2, ..., 1, 1, 1])

In [18]:
Y = tf.keras.utils.to_categorical(
    Y,
    num_classes = num_classes
)
Y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [19]:
train_df['sentence_no_punctuation'] = train_df['Body'].str.replace('[^\w\s]','')
train_df['sentence_no_punctuation'] = train_df["sentence_no_punctuation"].fillna("fillna")

<ipython-input-19-4dd1b29e7ba1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['sentence_no_punctuation'] = train_df['Body'].str.replace('[^\w\s]','')
<ipython-input-19-4dd1b29e7ba1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['sentence_no_punctuation'] = train_df["sentence_no_punctuation"].fillna("fillna")


In [20]:
train_df['sentence_no_punctuation'] = train_df['sentence_no_punctuation'].apply(remove_general_stopwords)

In [21]:
train_df['size'] = train_df['sentence_no_punctuation'].apply(lambda x:len(x.split()))

In [22]:
train_df['size'].describe()

,size
count,5438.000000
mean,237.010482
std,305.096537
min,1.000000
25%,86.000000
50%,153.000000
75%,267.750000
max,4437.000000


In [23]:
max_features = 10000
maxlen = 300

In [24]:
tok = tf.keras.preprocessing.text.Tokenizer(num_words = max_features)

In [25]:
tok.fit_on_texts(list(train_df['sentence_no_punctuation']))

In [26]:
print(len(tok.word_index))
vocab_size = len(tok.word_index)+1

56461


In [27]:
train_df = tok.texts_to_sequences(list(train_df['sentence_no_punctuation']))
train_df = tf.keras.preprocessing.sequence.pad_sequences(train_df, maxlen = maxlen)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(train_df, Y, test_size=0.1, random_state=42)

In [30]:
embedding_dim = 25

In [31]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim = vocab_size,
                            output_dim = embedding_dim,
                            input_length = maxlen),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6339 - loss: 0.8219
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9790 - loss: 0.0782
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9978 - loss: 0.0223
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9994 - loss: 0.0086
Epoch 5/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9995 - loss: 0.0066
Epoch 6/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0025
Epoch 7/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9997 - loss: 0.0033
Epoch 8/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9998 - loss: 0.0018
Epoch 9/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9999 - loss: 0.0012
Epoch 10/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9998 - loss: 0.0017


In [35]:
data = np.array(model.layers[0].get_weights())

In [36]:
data.shape

(1, 56462, 25)

In [37]:
data = data.reshape(data.shape[0]*data.shape[1], data.shape[2])

In [38]:
data = pd.DataFrame(data)

In [39]:
data.shape

(56462, 25)

In [40]:
data

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.013185,-0.005304,-0.008115,0.035271,-0.060813,0.015849,0.005112,0.000446,0.027801,-0.024830,...,-0.006753,0.032894,0.009663,0.032251,-0.045679,0.019706,-0.007442,0.036010,0.016372,0.006649
1,-0.044418,0.054093,-0.087914,0.069529,-0.002868,-0.051958,-0.045268,-0.045346,0.042967,0.014696,...,0.005856,-0.044298,0.146855,0.074566,0.002251,0.017303,-0.004208,0.029536,0.071981,0.037391
2,-0.087501,0.073788,0.145577,-0.187100,-0.041562,-0.071208,0.209420,-0.045625,0.048791,-0.135167,...,0.089273,0.051689,-0.011733,0.071883,0.162733,0.097391,0.041975,0.037094,0.077317,-0.107958
3,-0.134424,0.000755,0.078495,-0.042221,-0.008830,0.132120,0.093594,-0.073097,-0.003708,-0.009977,...,0.013991,-0.161679,-0.029438,-0.008687,0.015029,0.115170,0.045009,0.018055,0.149121,-0.005999
4,-0.055204,0.108554,0.117368,-0.109289,-0.024561,-0.073077,0.042544,-0.073140,-0.050280,-0.061414,...,0.135423,-0.006071,-0.075293,-0.045778,0.160463,0.002334,0.076176,-0.014442,0.067333,-0.130324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56457,0.032137,0.003761,0.007211,-0.011600,-0.047966,-0.007174,-0.042720,0.031231,0.035571,-0.001810,...,0.037044,-0.020310,-0.023330,0.006011,-0.034014,0.036877,0.028987,-0.031570,0.000073,-0.043863
56458,0.036984,-0.001206,0.000880,0.044966,-0.022812,0.015076,-0.000078,0.031213,-0.030775,-0.017417,...,-0.020052,-0.048833,0.048613,0.012156,0.017439,-0.037277,0.008956,0.025472,0.016253,0.013005
56459,-0.039707,0.027956,0.023550,-0.045412,0.002014,0.028977,0.043409,-0.020183,0.005087,-0.011066,...,0.048574,0.012724,-0.010248,-0.039977,-0.007866,0.009109,-0.029950,0.039997,0.032723,-0.038932
56460,-0.030088,-0.007342,0.025234,-0.023821,-0.034882,-0.042455,-0.027588,0.046593,0.032560,0.049188,...,-0.005566,-0.005758,-0.032540,0.028261,-0.044404,-0.025095,0.015831,-0.038658,0.013082,-0.037192


In [41]:
data_path = "/content/drive/MyDrive/Colab Notebooks/NLP/2/08-Word Cluste/data.csv"
data[1:].to_csv(data_path, sep='\t', index=False, header=False)

In [42]:
tok.word_index

{'ایران': 1,
 'کشور': 2,
 'است': 3,
 'کرد': 4,
 'گفت': 5,
 'اسلامی': 6,
 'باشگاه': 7,
 'جوان': 8,
 'خبرنگاران': 9,
 'گروه': 10,
 'مردم': 11,
 'انقلاب': 12,
 'انتهای': 13,
 'گزارش': 14,
 'مجلس': 15,
 'رئیس': 16,
 'سیاسی': 17,
 'اقتصادی': 18,
 'دولت': 19,
 'است،': 20,
 'بازار': 21,
 'افزود': 22,
 'خبرنگار': 23,
 'ملی': 24,
 'حضور': 25,
 'حوزه': 26,
 'شد': 27,
 'قیمت': 28,
 'شورای': 29,
 'سازمان': 30,
 'وزیر': 31,
 'آمریکا': 32,
 'وزارت': 33,
 'داد': 34,
 'آن\u200cها': 35,
 'فوتبال': 36,
 'جمهوری': 37,
 'توجه': 38,
 'روز': 39,
 '۲': 40,
 'اظهار': 41,
 'موضوع': 42,
 'تهران': 43,
 'تاکید': 44,
 'تولید': 45,
 'گذشته': 46,
 'بازی': 47,
 'توسعه': 48,
 'قانون': 49,
 'می\u200cشود': 50,
 'تصریح': 51,
 'اعلام': 52,
 'تومان': 53,
 'پیام': 54,
 'امام': 55,
 'درصد': 56,
 'ایجاد': 57,
 'بودجه': 58,
 'برنامه': 59,
 'استقلال': 60,
 'شرایط': 61,
 'بود': 62,
 'منطقه': 63,
 'نظام': 64,
 'بخوانید': 65,
 'میلیون': 66,
 'خارجی': 67,
 'طرح': 68,
 'لیگ': 69,
 'ملت': 70,
 'کاهش': 71,
 'برگزار': 72,
 'شود': 73,
 

In [43]:
meta_data = pd.DataFrame.from_dict(list(tok.word_index))

In [44]:
meta_data.shape

(56461, 1)

In [45]:
len(tok.word_index)

56461

In [50]:
meta_data.columns = ['word']

In [51]:
len(meta_data)

56461

In [52]:
meta_data_path = "/content/drive/MyDrive/Colab Notebooks/NLP/2/08-Word Cluste/meta_data.csv"
meta_data['word'].to_csv(meta_data_path, index=False, header=False)

In [53]:
meta_data

,word
0,ایران
1,کشور
2,است
3,کرد
4,گفت
...,...
56456,بروزخلاقیت‌های
56457,مغذی
56458,شهرداری‌هاست
56459,امیدیه


http://projector.tensorflow.org/